Need to figure out how to speed up things like perch. Let's do some basic benchmarking to openvino can speed things up by an order of magnitude.

In [99]:
import datetime
import tqdm

print(datetime.datetime.now().isoformat())

2025-06-05T20:45:28.184902


In [3]:
import bioacoustics_model_zoo as bmz

In [ ]:
from pathlib import Path

audio = sorted(Path("~/scratch/birdclef/test_soundscape").expanduser().glob("*.ogg"))[
    :2
]
audio

[PosixPath('/storage/home/hcoda1/8/amiyaguchi3/scratch/birdclef/test_soundscape/H02_20230420_074000.ogg'),
 PosixPath('/storage/home/hcoda1/8/amiyaguchi3/scratch/birdclef/test_soundscape/H02_20230420_112000.ogg')]

In [5]:
perch = bmz.list_models()["Perch"]()
perch

/storage/home/hcoda1/8/amiyaguchi3/clef/birdclef-2025/.venv/lib/python3.10/site-packages/tensorflow_hub/__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


/storage/home/hcoda1/8/amiyaguchi3/clef/birdclef-2025/.venv/lib/python3.10/site-packages/opensoundscape/ml/cnn.py:599: UserWarning: 
                    This architecture is not listed in opensoundscape.ml.cnn_architectures.ARCH_DICT.
                    It will not be available for loading after saving the model with .save() (unless using pickle=True). 
                    To make it re-loadable, define a function that generates the architecture from arguments: (n_classes, n_channels) 
                    then use opensoundscape.ml.cnn_architectures.register_architecture() to register the generating function.

                    The function can also set the returned object's .constructor_name to the registered string key in ARCH_DICT
                    to avoid this warning and ensure it is reloaded correctly by opensoundscape.ml.load_model().

                    See opensoundscape.ml.cnn_architectures module for examples of constructor functions
                    
  warnings.wa

Perch(
  (network): MLPClassifier(
    (hidden_layers): Sequential()
    (classifier): Linear(in_features=1280, out_features=10932, bias=True)
  )
  (loss_fn): BCEWithLogitsLoss_hot()
)

In [6]:
# NOTE: there is a warmup period for the model...
perch.predict(audio[0:1])
%time _ = perch.predict(audio)

  0%|          | 0/12 [00:00<?, ?it/s]

I0000 00:00:1749167273.520062 2333959 service.cc:146] XLA service 0x1ca947b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1749167273.520123 2333959 service.cc:154]   StreamExecutor device (0): Host, Default Version
2025-06-05 19:47:53.890956: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
W0000 00:00:1749167273.904441 2333959 assert_op.cc:38] Ignoring Assert operator jax2tf_infer_fn_/assert_equal_1/Assert/AssertGuard/Assert
I0000 00:00:1749167277.899508 2333959 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


  0%|          | 0/24 [00:00<?, ?it/s]

CPU times: user 34.1 s, sys: 98.3 ms, total: 34.2 s
Wall time: 31.3 s


In [107]:
scratch_root = Path("~/scratch/birdclef").expanduser()

for batch in perch.train_dataloader(audio, batch_size=2):
    break
batch[0].shape
batch

(array([[-0.00750089,  0.0125857 ,  0.00266587, ...,  0.00353971,
          0.01147204,  0.00857955],
        [ 0.00235424,  0.01088984, -0.0007503 , ...,  0.00444742,
         -0.00075508,  0.00074249]], dtype=float32),
 array([], shape=(2, 0), dtype=bool))

In [105]:
help(perch.train_dataloader)

Help on method train_dataloader in module opensoundscape.ml.cnn:

train_dataloader(samples, bypass_augmentations=False, collate_fn=<function collate_audio_samples at 0x7ffe655f4430>, **kwargs) method of bioacoustics_model_zoo.perch.Perch instance
    generate dataloader for training
    
    train_loader samples batches of images + labels from training set
    
    Args: see self.train_dataloader_cls docstring for arguments
        **kwargs: any arguments to pass to the DataLoader __init__
        Note: some arguments are fixed and should not be passed in kwargs:
        - shuffle=True: shuffle samples for training
        - bypass_augmentations=False: apply augmentations to training samples



In [ ]:
import tensorflow as tf

display(perch.tf_model.signatures.keys())
input_layer = tf.keras.layers.Input(
    shape=(160000,), dtype=tf.float32, name="waveform_input"
)
embedding_fn = perch.tf_model.signatures["serving_default"]
embedding_output = tf.keras.layers.Lambda(
    lambda x: embedding_fn(inputs=x)["embedding"],
    name="embedding",
    output_shape=embedding_fn.structured_outputs["embedding"].shape,
)(input_layer)
core_embedding_model = tf.keras.Model(
    inputs=input_layer,
    outputs=embedding_output,
    name="embedding",
)
core_embedding_model.summary()

KeysView(_SignatureMap({'serving_default': <ConcreteFunction (*, inputs: TensorSpec(shape=(None, 160000), dtype=tf.float32, name='inputs')) -> Dict[['order', TensorSpec(shape=(None, 41), dtype=tf.float32, name='order')], ['embedding', TensorSpec(shape=(None, 1280), dtype=tf.float32, name='embedding')], ['family', TensorSpec(shape=(None, 249), dtype=tf.float32, name='family')], ['frontend', TensorSpec(shape=(None, 500, 160), dtype=tf.float32, name='frontend')], ['genus', TensorSpec(shape=(None, 2333), dtype=tf.float32, name='genus')], ['label', TensorSpec(shape=(None, 10932), dtype=tf.float32, name='label')]] at 0x7FFFE411B310>}))

Model: "embedding"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ waveform_input (InputLayer)     │ (None, 160000)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Lambda)              │ (None, None, 1280)     │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
class PerchEmbeddingExtractor(tf.Module):
    def __init__(self, perch_model_object):
        super().__init__()
        self.original_model = perch_model_object

    @tf.function(
        input_signature=[tf.TensorSpec(shape=[None, 160000], dtype=tf.float32)]
    )
    def __call__(self, waveform):
        """
        This function defines the new, pruned computation graph.
        """
        output_dict = self.original_model.signatures["serving_default"](inputs=waveform)
        return {"embedding": output_dict["embedding"]}


# 3. Instantiate our new, clean module
pruned_model = PerchEmbeddingExtractor(perch.tf_model)
tf.saved_model.save(pruned_model, scratch_root / "compiled/perch_embedding_extractor")

INFO:tensorflow:Assets written to: /storage/home/hcoda1/8/amiyaguchi3/scratch/birdclef/compiled/perch_embedding_extractor/assets


INFO:tensorflow:Assets written to: /storage/home/hcoda1/8/amiyaguchi3/scratch/birdclef/compiled/perch_embedding_extractor/assets


In [72]:
converter = tf.lite.TFLiteConverter.from_keras_model(perch.tf_model)
# converter = tf.lite.TFLiteConverter.from_saved_model(
#     (scratch_root / "compiled/perch_embedding_extractor").as_posix()
# )
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TFLite native ops.
    tf.lite.OpsSet.SELECT_TF_OPS,  # Enable TF ops as a fallback.
]
tflite_model = converter.convert()
with (scratch_root / "compiled/perch.tflite").open("wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpkpo5umb9/assets


INFO:tensorflow:Assets written to: /tmp/tmpkpo5umb9/assets
W0000 00:00:1749170228.875034 2333959 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1749170228.875061 2333959 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2025-06-05 20:37:08.875300: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpkpo5umb9
2025-06-05 20:37:08.898063: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-06-05 20:37:08.898093: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpkpo5umb9
2025-06-05 20:37:09.181191: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-06-05 20:37:10.278259: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpkpo5umb9
2025-06-05 20:37:10.729380: I tensorflow/cc/saved_model/loader.cc:462] SavedModel load for tags { serve }; Status: success: OK. Took 185408

In [79]:
batch[0].shape

(1, 160000)

In [101]:
import numpy as np

# does this work any better than the original model?
interpreter = tf.lite.Interpreter(
    model_path=(scratch_root / "compiled/perch.tflite").as_posix()
)
interpreter.allocate_tensors()  # This is a mandatory step


def run_predict(interpreter, dataloader):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    res = []
    for batch in tqdm.tqdm(dataloader):
        interpreter.set_tensor(input_details[0]["index"], batch[0])
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[1]["index"])
        res.append(output_data)
    return np.stack(res)


%time _ = run_predict(interpreter, perch.train_dataloader(audio))

2025-06-05 20:47:03.848530: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_inter_op_parallelism which is not in the op definition: Op<name=Transpose; signature=x:T, perm:Tperm -> y:T; attr=T:type; attr=Tperm:type,default=DT_INT32,allowed=[DT_INT32, DT_INT64]> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node Transpose}}
100%|██████████| 24/24 [00:03<00:00,  7.09it/s]

CPU times: user 3.38 s, sys: 41 ms, total: 3.42 s
Wall time: 3.4 s


In [ ]:
! python -m tf2onnx.convert \
    --tflite {scratch_root / "compiled/perch.tflite"} \
    --output {scratch_root / "compiled/perch.onnx"} \
    --opset 18

2025-06-05 20:31:41.691339: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-05 20:31:41.709347: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-05 20:31:41.731101: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-05 20:31:41.737690: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-05 20:31:41.754000: I tensorflow/core/platform/cpu_feature_guar

In [71]:
import openvino as ov

# ov.convert_model(perch.tf_model, example_input=batch[0])
ove_model = ov.convert_model((scratch_root / "compiled/perch.tflite").as_posix())
# No translator found for FlexTranspose node.

OpConversionFailure: Check 'translate_map.count(decoder->get_op_type())' failed at src/frontends/tensorflow_lite/src/frontend.cpp:242:
FrontEnd API failed with OpConversionFailure:
No translator found for FlexTranspose node.
